# 상태 관리

<br>

<hr>

<br>

## 01. 상태 관리의 이해
- 웹 어플리케이션이 복잡해지면서 상태 관리 방식이 성능과 유지보수에 큰 영향
- React 애플리케이션은 사용자의 상호작용에 따라 UI가 변화하고 데이터를 실시간으로 처리해야 하며, 이러한 동작은 모두 상태를 기반으로 이루어짐

<br>

### 로컬 상태 관리
- **`useState`나 `useReducer`와 같은 훅을 사용해 각 컴포넌트 내부에 상태를 관리하는 방식**
  
- 상태는 해당 컴포넌트에만 국한되며, 다른 컴포넌트와 공유하려면 `props`를 통해 전달
  
  $\rightarrow$ 컴포넌트 구조가 복잡해질수록 이러한 상태 전달이 번거로워지고 관리도 어려워짐

<br>

- **컴포넌트를 분리했을 때 가장 중요한 점은 상태를 부모 컴포넌트(`App`)에 두고, 이를 `props`를 통해 자식 컴포넌트에 전달한다는 점**
  
  $\rightarrow$ **상태 끌어올리기(State Lifting)**

  - `App` 컴포넌트는 상태와 상태 변경함수를 관리
  - 자식 컴포넌트는  상태 값을 `props`로 전달받아 화면에 출력 (상태를 업데이트)
  
  $\rightarrow$ **부모가 상태를 소유하고, 자식은 `props`를 통해 상태를 사용**

- **로컬 상태 관리에서는 부모 $\rightarrow$ 자식 방향으로만 데이터 전달이 가능하며,**
  
  **형제 컴포넌트 간에는 직접적으로 상태 공유 불가**
  
  $\rightarrow$ 컴포넌트 구조가 단순한 소규모 애플리케이션에 적합

<br>

### 전역 상태 관리
- **상태를 애플리케이션 전체에서 공유 가능한 형태로 관리**
- 여러 컴포넌트가 전역 상태에 직접 접근하고 값을 수정할 수 있으므로, 일일히 `props`를 통해 전달할 필요가 없음

<br>

- **프레젠테이셔널 컴포넌트 (Presentational Component)** : **부모로부터 단순히 상태 값을 받아 화면에 표시만 하는 역할**
  - 상태 변경은 하지 않음
- **컨테이너 컴포넌트 (Container Component) : 부모로부터 전달받은 `props`를 직접 사용하지 않고, 하위 컴포넌트에 다시 전달만하는 역할**
  - 컴포넌트 구조가 깊어질수록 `props` 전달이 반복

<br>

<hr>

<br>

## 02. Context API
- Context API는 컴포넌트 트리 내에서 상태나 값을 전역적으로 공유할 수 있게 하는 기능
  - **컨텍스트 : 프로그래밍에서 컴포넌트들이 공통으로 사용하는 데이터의 흐름이나 의미를 담는 공간**
  - **API : 다양한 소프트웨어 컴포넌트가 상호작용할 수 있게 하는 인터페이스**

<br>

- **React에서 컨텍스트는 여러 컴포넌트를 하나의 환경으로 묶어 이들 사이에서 상태나 데이터를 `props` 없이 직접 공유할 수 있도록 해주는 구조**
- **하나의 상태를 공유하는 모든 컨텍스트 $\rightarrow$ 컨텍스트 범위 (Context Range)**
  
  $\rightarrow$ 하나의 컨텍스트로 묶인 컴포넌트들은 같은 환경 안에 있다고 간주하며, 해당 컨텍스트가 제공하는 상태나 값을 `props`없이 직접 사용

<br>

### 컨텍스트 객체 생성
- **컨텍스트 객체 : 데이터를 전역으로 공유하기 위한 매개체 역할**
- **내부에 Provider 컴포넌트를 포함해 이를 사용해서 전역 상태를 어느 범위의 컴포넌트에서 사용할 지 지정 가능**
  - Provider : 컨텍스트의 데이터를 하위 컴포넌트에 공급하는 컴포넌트
  - Proivder로 감싼 컴포넌트들은 `props` 없이도 컨텍스트 값을 사용 가능

<br>

- **`createContext()`**
    - `<DataType>` : 컨텍스트에서 공유할 데이터의 타입을 제네릭 타입으로 지정
    - `defaultValue` : 컨텍스트의 초깃값

```ts
const SomeContext = createContext<DataType>(defaultValue);
```

<br>

- `src/contexts/CountContext.ts`

```ts
import { createContext } from "react";

interface CountContextType {
  count: number;
  increment: () => void;
  decrement: () => void;
  reset: () => void;
}

export const CountContext = createContext<CountContextType | null>(null);
```

<br>

### Provider로 컨텍스트 범위 지정
- 어떤 컴포넌트들이 전역 상태를 공유할지 지정
    - `value` : 하위 컴포넌트에 전달할 데이터 (상태, 함수 등)
    - `공유할 컴포넌트` : 전달할 데이터를 사용할 컴포넌트

```ts
<컨텍스트_객체 value={공유할_데이터}>
    공유할_컴포넌트
</ 컨텍스트_객체>
```

<br>

- `src/providers/CountProviders.tsx`

```ts
import { useState } from "react";
import { CountContext } from "../contexts/CountContext";

export default function CountProvider({
  children,
}: {
  children: React.ReactNode;
}) {
  const [count, setCount] = useState(0);
  const increment = () => setCount((count) => count + 1);
  const decrement = () => setCount((count) => count - 1);
  const reset = () => setCount(0);
  return (
    <>
      <CountContext value={{ count, increment, decrement, reset }}>
        {children}
      </CountContext>
    </>
  );
}
```

<br>

### `useContext` 커스텀 훅
- 컨텍스트로 공유되는 데이터를 쉽게 사용하기 위해 별도의 커스텀 훅 생성
- `useContext`는 Context API로 공유된 값을 하위 컴포넌트에서 직접 가져올 수 있게 해주는 훅
    - `value` : 컨텍스트에서 제공한 데이터
    - `SomeContext` : `createContext()`로 생성한 컨텍스트 객체

```ts
const value = useContext(SomeContext);
```

<br>

- `src/hooks/useCountContext.ts`

```ts
import { useContext } from "react";
import { CountContext } from "../contexts/CountContext";

export function useCountContext() {
  const context = useContext(CountContext);
  if (!context) {
    throw new Error(
      "useCountContext는 CountContext로 감싼 컴포넌트 안에서만 호출할 수 있습니다."
    );
  }
  return context;
}
```

<br>

### 컨텍스트로 공유되는 전역상태 사용
- `App.tsx`
  - Provider로 감싼 범위 안에서는 `props`를 사용하지 않아도 됨

```ts
import Count from "./components/Count";
import CountOutsideDisplay from "./components/CountOutsideDisplay";
import CountProvider from "./providers/CountProvider";

export default function App() {
  return (
    <>
      <CountProvider>
        <Count />
        <CountOutsideDisplay />
      </CountProvider>
    </>
  );
}
```

<br>

- `src/components/Count.tsx`

```ts
import CountDisplay from "./CountDisplay";
import CountButtons from "./CountButtons";

export default function Count() {
  return (
    <>
      <CountDisplay />
      <CountButtons />
    </>
  );
}
```

<br>

- `src/components/CountDisplay.tsx`

```ts
import { useCountContext } from "../hooks/useCountContext";

export default function CountDisplay() {
  const { count } = useCountContext();
  return <h1>Count: {count}</h1>;
}
```

<br>

- `src/components/CountButtons.tsx`

```ts
import { useCountContext } from "../hooks/useCountContext";

export default function CountButtons() {
  const { increment, decrement, reset } = useCountContext();
  return (
    <>
      <button onClick={decrement}>감소</button>
      <button onClick={reset}>초기화</button>
      <button onClick={increment}>증가</button>
    </>
  );
}
```

<br>

- `src/components/CountOutsideDisplay.tsx`

```ts
import { useCountContext } from "../hooks/useCountContext";

export default function CountOutsideDisplay() {
  const { count } = useCountContext();
  return <h1>Outside Count: {count}</h1>;
}
```

<br>

### 렌더링 최적화
- Context API를 사용하면 컴포넌트 트리 전체에 상태를 전역적으로 공유 가능
- 컨텍스트에서 제공하는 데이터르 커스텀 훅으로 직접 사용하는 컴포넌트는 상태 변경시 리렌더링되므로 불필요한 리렌더링을 줄이는 데도 도움
- **Context API는 데이터를 직접 사용하지 않는 중간 컴포넌트는 렌더링에 영향을 받지 않음**

<br>

### 컨텍스트 중첩 사용
- **Context API는 여러 개의 컨텍스트 객체를 생성해 컨텍스트 범위를 중첩해 구성할 수 있음**

<img src='img/1202.png' width=600>

<br>

- `src/contexts/AuthContext.ts`

```ts
import {createContext} from 'react';

interface AuthContextType {
  isLogin: boolean;
  login: () => void;
  logout: () => void;
}

export const AuthContext = createContext<AuthContextType | null>(null);
```

<br>

- `src/components/AuthProvider.tsx`
- 생성한 `AuthContext`를 기반으로 로그인 상태를 전역으로 관리할 수 있는 `AuthProvider` 컴포넌트를 정의

```ts
import { useState } from "react";
import { AuthContext } from "../contexts/AuthContext";

export default function AuthProvider({
  children,
}: {
  children: React.ReactNode;
}) {
  const [isLogin, setIsLogin] = useState(false);
  const login = () => setIsLogin(true);
  const logout = () => setIsLogin(false);

  return (
    <AuthContext value={{ isLogin, login, logout }}>{children}</AuthContext>
  );
}
```

<br>

- `src/hooks/useAuthContext.ts`

```ts
import { useContext } from "react";
import { AuthContext } from "../contexts/AuthContext";

export function useAuthContext() {
  const context = useContext(AuthContext);
  if (!context) {
    throw new Error(
      "useAuthContext AuthProvider로 감싼 컴포넌트 안에서만 호출할 수 있습니다."
    );
  }
  return context;
}
```

<br>

- `useAuthContext`를 사용하는 `Auth` 컴포넌트
- `src/components/Auth.tsx`
  - `isLogin`, `login`, `logout`을 가져와 현재 로그인 상태를 화면에 표시하고, 버튼을 사용해 상태를 변경

```ts
import { useAuthContext } from "../hooks/useAuthContext";

export default function Auth() {
  console.log("Auth rendering");
  const { isLogin, login, logout } = useAuthContext();
  return (
    <>
      <h1>login: {isLogin.toString()}</h1>
      <button onClick={login}>로그인</button>
      <button onClick={logout}>로그아웃</button>
    </>
  );
}
```

<br>

### Context API 사용 시 주의사항
- **같은 컨텍스트 범위 안에 있는 컴포넌트끼리만 데이터 공유 가능**

<br>

### `use` 훅으로 Context API 사용 (React 19 이후)
- **React 19 이후에서는 `useContext`훅을 대체할 수 있는 새로운 `use`훅이 도입**
  - `useContext`훅과 기능은 거의 동일
  - **항상 컴포넌트의 최상위에서만 호출해야 하는 `useContext` 훅과 달리, `use`훅은 `if`문 등의 조건문 안에서도 사용 가능**
- **React 19 이상에서는 `useContext`대신 `use` 훅을 사용하는 것이 바람직**

```ts
const value = use(SomeContext);
```

<br>

- `src/hooks/useAuthContext.ts`

```ts
import { use } from "react";
import { AuthContext } from "../contexts/AuthContext";

export function useAuthContext() {
  if (true) {
    const context = use(AuthContext);
    if (!context) {
      throw new Error(
        "useAuthContext AuthProvider로 감싼 컴포넌트 안에서만 호출할 수 있습니다."
      );
    }
    return context;
  }
}
```

<br>

<hr>
